In [195]:
import os

In [196]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [197]:
#os.chdir('../')

In [198]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [199]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    trained_model_path: Path
    updated_base_model_path: Path
    params_batch_size: int
    params_epochs: int
    target_column: str

In [200]:
from TimeSeriesProject .constants import *
from TimeSeriesProject.utils.common import read_yaml, create_directories

In [201]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            params_batch_size=params.BATCH_SIZE,
            params_epochs=params.EPOCHS,
            target_column = schema.name
            
        )

        return model_trainer_config

In [202]:
import os
import joblib
import pandas as pd
from TimeSeriesProject import logger
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [203]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_data.drop(['RUL', 'id'], axis=1, inplace=True)
        test_data.drop(['RUL', 'id'], axis=1, inplace=True)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # train_x = train_x.to_numpy()
        # test_x = test_x.to_numpy()
        # train_y = train_y.to_numpy()
        # test_y = test_y.to_numpy()

        # train_x = train_x.reshape(-1, 9, 1)
        # test_x = test_x.reshape(-1, 9, 1)

        train_x_lstm = train_x.values.reshape((train_x.shape[0], train_x.shape[1], 1))
        test_x_lstm = test_x.values.reshape((test_x.shape[0], test_x.shape[1], 1))



        self.model.fit(
            train_x_lstm, train_y,
            batch_size=self.config.params_batch_size,
            epochs=self.config.params_epochs,
            verbose=1  
        )

        #joblib.dump(self.model, os.path.join(self.config.root_dir, self.model))

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
         )



In [204]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.get_base_model()
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-15 01:37:15,354: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-15 01:37:15,354: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-15 01:37:15,369: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-15 01:37:15,382: INFO: common: created directory at: artifacts]
[2023-09-15 01:37:15,383: INFO: common: created directory at: artifacts/model_trainer]
Epoch 1/10
484/484 [==============================] - 3s 4ms/step - loss: 0.2055 - accuracy: 0.9204
Epoch 2/10
484/484 [==============================] - 1s 3ms/step - loss: 0.1301 - accuracy: 0.9463
Epoch 3/10
484/484 [==============================] - 1s 3ms/step - loss: 0.1274 - accuracy: 0.9455
Epoch 4/10
484/484 [==============================] - 1s 3ms/step - loss: 0.1238 - accuracy: 0.9477
Epoch 5/10
484/484 [==============================] - 1s 3ms/step - loss: 0.1219 - accuracy: 0.9479
Epoch 6/10
484/484 [==============================] - 1s 3ms/step - loss: 0.1215 

f:\Time Series Data Analysis (Upwork)\end-to-end-time-series-data-analysis-with-mlflow\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
